## 如果我想下载10张图片，每隔0.5秒下载一次，也就是：

以下内容只对IO密集时起作用，对CPU密集时使用线程只会更慢，那时应该使用进程，但这里还未写。

In [64]:
import time
import datetime

def downloadImage(threadName, time_step, N):
    while N:
        time.sleep(time_step)
        ctime = datetime.datetime.now()
        print(f'工人:{threadName:^7} 正在下载: {ctime}')
        N -= 1
    return threadName
        
N = 10
threadName = 'YKuny'
downloadImage(threadName, 0.5, N)

工人: YKuny  正在下载: 2019-08-18 23:50:05.126603
工人: YKuny  正在下载: 2019-08-18 23:50:05.627349
工人: YKuny  正在下载: 2019-08-18 23:50:06.127671
工人: YKuny  正在下载: 2019-08-18 23:50:06.628941
工人: YKuny  正在下载: 2019-08-18 23:50:07.129635
工人: YKuny  正在下载: 2019-08-18 23:50:07.630389
工人: YKuny  正在下载: 2019-08-18 23:50:08.131230
工人: YKuny  正在下载: 2019-08-18 23:50:08.631692
工人: YKuny  正在下载: 2019-08-18 23:50:09.132482
工人: YKuny  正在下载: 2019-08-18 23:50:09.633223


'YKuny'

恩，可以看到，**很慢**.

如果这时，可以多一个进程一起来下载的话:

In [9]:
import threading

class MyThread(threading.Thread):
    def __init__(self, threadID, threadName, N):
        super().__init__()
        self.threadID = threadID
        self.threadName = threadName
        self.N = N
    
    def run(self):
        print(f"{self.threadName}: 开始线程")
        downloadImage(self.threadName, 0.5, self.N)
        print(f"{self.threadName}: 退出线程")

In [14]:
thread1 = MyThread(1, '小1', 5)
thread2 = MyThread(2, '小2', 5)

# 线程调用start()方法的时候，就会去执行run方法
thread1.start()
thread2.start()

# join()方法是为了让线程执行完再终止程序
thread1.join()
thread2.join()

小1: 开始线程
小2: 开始线程
工人  小1    正在下载:, 2019-08-18 22:55:40.914569
工人  小2    正在下载:, 2019-08-18 22:55:40.916006
工人  小1    正在下载:, 2019-08-18 22:55:41.415592
工人  小2    正在下载:, 2019-08-18 22:55:41.416800
工人  小1    正在下载:, 2019-08-18 22:55:41.916586
工人  小2    正在下载:, 2019-08-18 22:55:41.917993
工人  小1    正在下载:, 2019-08-18 22:55:42.417601
工人  小2    正在下载:, 2019-08-18 22:55:42.418832
工人  小1    正在下载:, 2019-08-18 22:55:42.918489
小1: 退出线程
工人  小2    正在下载:, 2019-08-18 22:55:42.920525
小2: 退出线程


## 线程池
那么，如果我要创建多线程，这样一个一个添加进去是可行的，但如果需要创建的线程很多时，频繁创建线程和销毁线程是非常浪费资源的事情。  
所以我们有了线程池，线程池就可以重复利用线程：

In [18]:
from concurrent.futures import ThreadPoolExecutor

In [69]:
with ThreadPoolExecutor(max_workers=20) as executor:
    arg1 = ('t1', 0.5, 5)
    arg2 = ('t2', 0.5, 5)
    # executor.submit(fn, *args, **kwargs),其中*args是fn的参数
    task1 = executor.submit(downloadImage, *arg1)
    task2 = executor.submit(downloadImage, *arg2)
    print(task1.done())
    time.sleep(1)
    print(task1.done())
    


False
工人:  t2    正在下载: 2019-08-18 23:51:37.645140
工人:  t1    正在下载: 2019-08-18 23:51:37.646107
False
工人:  t1    正在下载: 2019-08-18 23:51:38.147260
工人:  t2    正在下载: 2019-08-18 23:51:38.147723
工人:  t1    正在下载: 2019-08-18 23:51:38.648853
工人:  t2    正在下载: 2019-08-18 23:51:38.650333
工人:  t1    正在下载: 2019-08-18 23:51:39.150491
工人:  t2    正在下载: 2019-08-18 23:51:39.151950
工人:  t1    正在下载: 2019-08-18 23:51:39.651932
工人:  t2    正在下载: 2019-08-18 23:51:39.653677


In [62]:
# 当然，用字典实现函数参数也一样
with ThreadPoolExecutor(max_workers=20) as executor: 
    dict1 = {'threadName': 't1', 'time_step':0.5, 'N':5}
    dict2 = {'threadName': 't2', 'time_step':0.5, 'N':5}

    task1 = executor.submit(downloadImage, **dict1)
    task2 = executor.submit(downloadImage, **dict2)

工人:  t1    正在下载: 2019-08-18 23:48:44.098747
工人:  t2    正在下载: 2019-08-18 23:48:44.102028
工人:  t1    正在下载: 2019-08-18 23:48:44.599709
工人:  t2    正在下载: 2019-08-18 23:48:44.602850
工人:  t1    正在下载: 2019-08-18 23:48:45.100473
工人:  t2    正在下载: 2019-08-18 23:48:45.103521
工人:  t1    正在下载: 2019-08-18 23:48:45.601182
工人:  t2    正在下载: 2019-08-18 23:48:45.604273
工人:  t1    正在下载: 2019-08-18 23:48:46.101949
工人:  t2    正在下载: 2019-08-18 23:48:46.104936
